In [1]:
from __future__ import print_function

import os.path
import pandas as pd
import numpy as np
import keras
from keras import applications, metrics, layers, models, regularizers, optimizers
from keras.applications import ResNet50, Xception, InceptionResNetV2
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from tqdm import tqdm_notebook
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from skimage.transform import resize
from sklearn.metrics import classification_report

/home/aminov/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv("dataset/Selfie-dataset/selfie_dataset.csv", sep= " ", index_col=False)

In [3]:
df.columns

Index(['name', 'score', 'partial_faces', 'is_female', 'baby', 'child',
       'teenager', 'youth', 'middle_age', 'senior', 'white', 'black', 'asian',
       'oval_face', 'round_face', 'heart_face', 'smiling', 'mouth_open',
       'frowning', 'wearing_glasses', 'wearing_sunglasses', 'wearing_lipstick',
       'tongue_out', 'duck_face', 'black_hair', 'blond_hair', 'brown_hair',
       'red_hair', 'curly_hair', 'straight_hair', 'braid_hair',
       'showing_cellphone', 'using_earphone', 'using_mirror', 'braces',
       'wearing_hat', 'harsh_lighting', 'dim_lighting'],
      dtype='object')

In [81]:
# кол-во улыбок
len(df[df['smiling'] == 1])

12207

In [82]:
# кол-во открытых ртов
len(df[df['mouth_open'] == 1]['name'])

1819

In [4]:
get_y = lambda x : x if x > 0 else 0
batch = 32
trg_size = 200
total_batch_size = int(len(df) / batch)
start_pos = 0
def train_generation():
    
    while True:
      global start_pos
      if start_pos >= total_batch_size:
        start_pos = 0
    
      X = np.empty((batch, trg_size, trg_size, 3))
      y_sm = np.empty((batch, 1))
      y_mo = np.empty((batch, 1))
        
      for i,(ind,row) in enumerate(df[start_pos:start_pos+batch].iterrows()):

          X[i] = np.array(load_img("dataset/Selfie-dataset/images/"+row['name']+".jpg", 
                                   target_size=(trg_size ,trg_size))) / 255.0

          y_sm[i] = get_y(row['smiling'])
          y_mo[i] = get_y(row['mouth_open'])
            
      start_pos+=batch
        
      yield X, [y_sm, y_mo]
#       yield X, y_mo

In [5]:
# обучение. Каждый раз создается новый экземляр InceptionResNetV2. Заведомо не использовал понижение lr 
# и прочие модные фишки т.к. предобученная и так норм. сходится
def one_model(name):
    base_model = InceptionResNetV2(input_shape=(None, None, 3), weights='imagenet')

    for l in base_model.layers:
        l.trainable = True

    x1 = base_model.layers[-2].output
    y1 = Dense(1, activation='sigmoid')(x1)

    x2 = base_model.layers[-2].output
    y2 = Dense(1, activation='sigmoid')(x2)

    model = Model([base_model.input], [y1, y2])    

    model.compile(loss=['mse','mse'], optimizer='adam', metrics=['accuracy'])

    model.fit_generator(train_generation(), 
                        steps_per_epoch=total_batch_size, 
                        epochs=3)
    
    model.save_weights(name+'.hdf5')
    

In [8]:

for itr in range(5):
    one_model(str(itr))

Epoch 1/3
1463/1463 [==============================] - 749s 512ms/step - loss: 0.0572 - dense_1_loss: 0.0419 - dense_2_loss: 0.0153 - dense_1_acc: 0.9502 - dense_2_acc: 0.9836
Epoch 2/3
1463/1463 [==============================] - 691s 472ms/step - loss: 0.0196 - dense_1_loss: 0.0149 - dense_2_loss: 0.0047 - dense_1_acc: 0.9830 - dense_2_acc: 0.9950
Epoch 3/3
1463/1463 [==============================] - 689s 471ms/step - loss: 0.0104 - dense_1_loss: 0.0075 - dense_2_loss: 0.0028 - dense_1_acc: 0.9920 - dense_2_acc: 0.9972
Epoch 1/3
1463/1463 [==============================] - 796s 544ms/step - loss: 0.0634 - dense_3_loss: 0.0460 - dense_4_loss: 0.0175 - dense_3_acc: 0.9453 - dense_4_acc: 0.9815
Epoch 2/3
1463/1463 [==============================] - 780s 533ms/step - loss: 0.0135 - dense_3_loss: 0.0091 - dense_4_loss: 0.0044 - dense_3_acc: 0.9907 - dense_4_acc: 0.9956
Epoch 3/3
1463/1463 [==============================] - 766s 524ms/step - loss: 0.0354 - dense_3_loss: 0.0254 - dense_4_l

In [5]:
def create_model():
    base_model = InceptionResNetV2(input_shape=(None, None, 3))
    x1 = base_model.layers[-2].output
    y1 = Dense(1, activation='sigmoid')(x1)

    x2 = base_model.layers[-2].output
    y2 = Dense(1, activation='sigmoid')(x2)
    model = Model([base_model.input], [y1, y2])
    
    return model
    
models = []
whts = ['2.hdf5', '3.hdf5']
for i in range(2):
    model = create_model()
    model.load_weights(whts[i])
    models.append( model )

In [6]:
new_model = create_model()

In [7]:
# Усреднение весов для 3-й сети
#
weights = [model.get_weights() for model in models]

new_weights = list()

for weights_list_tuple in zip(*weights):
#     new_weights.append([np.array(weights_).mean(axis=0) for weights_ in zip(*weights_list_tuple)])
    new_weights.append(np.array(weights_list_tuple).mean(axis=0))
    
new_model.set_weights(new_weights)

In [99]:
new_model.save_weights('mean_w.hdf5')

In [11]:
def get_image(name):
    img = load_img("dataset/Selfie-dataset/images/"+name+".jpg", target_size=(trg_size ,trg_size))
    return np.array(img) / 255.0

In [62]:
y_pred_smile_m1 = []
y_pred_mouth_m1 = []

y_pred_smile_m2 = []
y_pred_mouth_m2 = []

y_pred_smile_mean = []
y_pred_mouth_mean = []

for i,row in tqdm_notebook(df.iterrows()):
    img = get_image(row['name'])
    y1 = get_y(row['smiling'])
    y2 = get_y(row['mouth_open'])
      
    y_pred_m1 = models[0].predict(np.array([img]))
    y_pred_smile_m1.append(np.round(y_pred_m1[0][0])[0])
    y_pred_mouth_m1.append(np.round(y_pred_m1[1][0])[0])
    
    y_pred_m2 = models[0].predict(np.array([img]))
    y_pred_smile_m2.append(np.round(y_pred_m2[0][0])[0])
    y_pred_mouth_m2.append(np.round(y_pred_m2[1][0])[0])
    
    y_pred_mean = models[0].predict(np.array([img]))
    y_pred_smile_mean.append(np.round(y_pred_mean[0][0])[0])
    y_pred_mouth_mean.append(np.round(y_pred_mean[1][0])[0])
    

In [63]:
ddf = df
y_true_sm = [ float(get_y(row['smiling'])) for i,row in ddf.iterrows() ]
y_true_mo = [ float(get_y(row['mouth_open'])) for i,row in ddf.iterrows() ]

In [91]:
print('result model 1:')
print('class: smile/non smile: \n','-'*90)
print(classification_report(y_true_sm, y_pred_smile_m1, target_names=['smile', 'none_smile']) )
print('class: open mouth/close mouth: \n','-'*90)
print(classification_report(y_true_mo, y_pred_mouth_m1, target_names=['open', 'close']) )

result model 1:
class: smile/non smile: 
 ------------------------------------------------------------------------------------------
             precision    recall  f1-score   support

      smile       0.85      0.94      0.89     34629
 none_smile       0.75      0.53      0.62     12207

avg / total       0.82      0.83      0.82     46836

class: open mouth/close mouth: 
 ------------------------------------------------------------------------------------------
             precision    recall  f1-score   support

       open       0.96      1.00      0.98     45017
      close       0.64      0.03      0.06      1819

avg / total       0.95      0.96      0.94     46836



In [88]:
print('result model 2:')
print('class: smile/non smile: \n','-'*90)
print(classification_report(y_true_sm, y_pred_smile_m2, target_names=['smile', 'none_smile']) )
print('class: open mouth/close mouth: \n','-'*90)
print(classification_report(y_true_mo, y_pred_mouth_m2, target_names=['open', 'close']) )

result model 2:
class: smile/non smile: 
 ------------------------------------------------------------------------------------------
             precision    recall  f1-score   support

      smile       0.85      0.94      0.89     34629
 none_smile       0.75      0.53      0.62     12207

avg / total       0.82      0.83      0.82     46836

class: open mouth/close mouth: 
 ------------------------------------------------------------------------------------------
             precision    recall  f1-score   support

       open       0.96      1.00      0.98     45017
      close       0.64      0.03      0.06      1819

avg / total       0.95      0.96      0.94     46836



In [89]:
print('result model mean:')
print('class: smile/non smile: \n','-'*90)
print(classification_report(y_true_sm, y_pred_smile_mean, target_names=['smile', 'none_smile']) )
print('class: open mouth/close mouth: \n','-'*90)
print(classification_report(y_true_mo, y_pred_mouth_mean, target_names=['open', 'close']) )

result model mean:
class: smile/non smile: 
 ------------------------------------------------------------------------------------------
             precision    recall  f1-score   support

      smile       0.85      0.94      0.89     34629
 none_smile       0.75      0.53      0.62     12207

avg / total       0.82      0.83      0.82     46836

class: open mouth/close mouth: 
 ------------------------------------------------------------------------------------------
             precision    recall  f1-score   support

       open       0.96      1.00      0.98     45017
      close       0.64      0.03      0.06      1819

avg / total       0.95      0.96      0.94     46836



## ----------------------------------------------------------------------------------------------------------------------------------
## ------------------------------------------------------Use Model-------------------------------------------------------------

In [27]:
import glob

In [107]:
files = glob.glob('test/*/*', recursive=True)# Укажите здесь путь до изо-ий

In [108]:
files

['test/Adam_Scott/Adam_Scott_0001.jpg',
 'test/Adam_Scott/Adam_Scott_0002.jpg',
 'test/Adrian_Fernandez/Adrian_Fernandez_0001.jpg',
 'test/Adrien_Brody/Adrien_Brody_0011.jpg',
 'test/Adrien_Brody/Adrien_Brody_0007.jpg',
 'test/Adrien_Brody/Adrien_Brody_0010.jpg',
 'test/Adrien_Brody/Adrien_Brody_0006.jpg',
 'test/Adrien_Brody/Adrien_Brody_0002.jpg',
 'test/Adrien_Brody/Adrien_Brody_0003.jpg',
 'test/Adrien_Brody/Adrien_Brody_0001.jpg',
 'test/Adrien_Brody/Adrien_Brody_0008.jpg',
 'test/Adrien_Brody/Adrien_Brody_0005.jpg',
 'test/Adrien_Brody/Adrien_Brody_0004.jpg',
 'test/Adrien_Brody/Adrien_Brody_0012.jpg',
 'test/Adrien_Brody/Adrien_Brody_0009.jpg',
 'test/Alan_Greer/Alan_Greer_0001.jpg',
 'test/Aldo_Paredes/Aldo_Paredes_0001.jpg',
 'test/Aldo_Paredes/Aldo_Paredes_0002.jpg',
 'test/Abdullah_al-Attiyah/Abdullah_al-Attiyah_0002.jpg',
 'test/Abdullah_al-Attiyah/Abdullah_al-Attiyah_0001.jpg',
 'test/Abdullah_al-Attiyah/Abdullah_al-Attiyah_0003.jpg',
 'test/Alberto_Acosta/Alberto_Acosta_0

In [109]:
def get_print(file, y):
    w1 = 'нет улыбки'
    w2 = 'рот закрыт'
    
    if y[0][0][0] > 0.51:
        w1 = 'улыбка'
    if y[1][0][0] > 0.51:
        w2 = 'рот открыт'
    print(file,':\n',w1,', \n',w2)
    print('-'*90)

def create_model():
    base_model = InceptionResNetV2(input_shape=(None, None, 3))
    x1 = base_model.layers[-2].output
    y1 = Dense(1, activation='sigmoid')(x1)

    x2 = base_model.layers[-2].output
    y2 = Dense(1, activation='sigmoid')(x2)
    model = Model([base_model.input], [y1, y2])
    
    return model

In [6]:
def get_image(path_to_img):
    img = load_img(path_to_img)
    return np.array(img) / 255.0

In [7]:
model = create_model()

In [100]:
model.load_weights('3.hdf5')

In [110]:
for file in files:
    img = get_image(file)
    y = np.round(model.predict(np.array([img])))

    get_print(file,y)

test/Adam_Scott/Adam_Scott_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adam_Scott/Adam_Scott_0002.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adrian_Fernandez/Adrian_Fernandez_0001.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adrien_Brody/Adrien_Brody_0011.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adrien_Brody/Adrien_Brody_0007.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adrien_Brody/Adrien_Brody_0010.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adrien_Brody/Adrien_Brody_0006.jpg :
 улыбка , 

test/Albert_Costa/Albert_Costa_0003.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Albert_Costa/Albert_Costa_0002.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Akiko_Morigami/Akiko_Morigami_0001.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alan_Ball/Alan_Ball_0001.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alan_Ball/Alan_Ball_0002.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Ainsworth_Dyer/Ainsworth_Dyer_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alecos_Markides/Alecos_Markides_0001.jpg :
 улыбка 

test/Abdullah_Gul/Abdullah_Gul_0006.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Abdullah_Gul/Abdullah_Gul_0002.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Abdullah_Gul/Abdullah_Gul_0009.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Abdullah_Gul/Abdullah_Gul_0019.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Abdullah_Gul/Abdullah_Gul_0007.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Abdullah_Gul/Abdullah_Gul_0003.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Abdullah_Gul/Abdullah_Gul_0011.jpg :
 н

test/Adrian_McPherson/Adrian_McPherson_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adrian_McPherson/Adrian_McPherson_0002.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alain_Ducasse/Alain_Ducasse_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/AJ_Cook/AJ_Cook_0001.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adrian_Murrell/Adrian_Murrell_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Abba_Eban/Abba_Eban_0001.jpg :
 улыбка , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alan_Trammell/Alan_Trammell_0001.jpg 

test/Adrian_Annus/Adrian_Annus_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Aidan_Quinn/Aidan_Quinn_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Adam_Rich/Adam_Rich_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alberto_Fujimori/Alberto_Fujimori_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alberto_Fujimori/Alberto_Fujimori_0002.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Albert_Montanes/Albert_Montanes_0001.jpg :
 нет улыбки , 
 рот закрыт
------------------------------------------------------------------------------------------
test/Alastair_Campbell/Ala

In [86]:
import timeit

In [124]:
img = get_image('1.jpg')

In [127]:
%%time
y = model.predict(np.array([img]))

CPU times: user 68 ms, sys: 4 ms, total: 72 ms
Wall time: 65.2 ms


In [128]:
y

[array([[0.99999976]], dtype=float32), array([[0.00010956]], dtype=float32)]

## Вывод: